Dieses Skript berechnet die ersten Features und speichert sie ab. Dies ist ebenso ein erster Test für die geschriebene Methode zum Ausrechnen der features.

In [ ]:
from utils.training import calculate_features

features = ['mean', 'variance', 'std']
hdf5_feat = './features/features.hdf5'
hdf5_data = './labelled_data/labels.hdf5'

calculate_features(hdf5_feat, hdf5_data, features)

Calculating features for sessions:   0%|          | 0/268 [00:00<?, ?it/s]

Calculating features for channel A1-T3 in session aaaaaaju_s005_t000: 100%|██████████| 3/3 [00:00<00:00, 11.25it/s]


Calculating features for channel C3-CZ in session aaaaaaju_s005_t000: 100%|██████████| 3/3 [00:00<00:00, 18.47it/s]


Calculating features for channel C3-P3 in session aaaaaaju_s005_t000: 100%|██████████| 3/3 [00:00<00:00, 21.74it/s]


Calculating features for channel C4-P4 in session aaaaaaju_s005_t000: 100%|██████████| 3/3 [00:00<00:00, 15.32it/s]


Calculating features for channel C4-T4 in session aaaaaaju_s005_t000: 100%|██████████| 3/3 [00:00<00:00, 22.17it/s]


Calculating features for channel CZ-C4 in session aaaaaaju_s005_t000: 100%|██████████| 3/3 [00:00<00:00, 18.19it/s]


Calculating features for channel F3-C3 in session aaaaaaju_s005_t000: 100%|██████████| 3/3 [00:00<00:00, 10.94it/s]


Calculating features for channel F4-C4 in session aaaaaaju_s005_t000: 100%|██████████| 3/3 [00:00<

Diese drei Features haben 16 Minuten und 13 Sekunden gebraucht, um sie auszurechnen.

In [16]:
import h5py
import numpy as np

with h5py.File(hdf5_feat) as f, h5py.File(hdf5_data) as df:
    print(f["aaaaaall_s004_t000"]['A1-T3']['mean'][:])
    print(np.mean(df["aaaaaall_s004_t000"]['A1-T3']['data_windows'], axis=1))

[-0.21762188  0.66732749 -0.44213544 ...  0.81763384 -1.06459474
  0.30870412]
[-0.21762188  0.66732749 -0.44213544 ...  0.81763384 -1.06459474
  0.30870412]


Die Methode scheint funktioniert zu haben. Im folgenden werden noch weitere Features generiert.

In [1]:
from utils.training import calculate_features

features = ['ptp_amp', 'skewness', 'kurtosis', 'quantile']
hdf5_feat = './features/features.hdf5'
hdf5_data = './labelled_data/labels.hdf5'

calculate_features(hdf5_feat, hdf5_data, features)

Calculating features for sessions:  83%|████████▎ | 222/268 [15:48<04:51,  6.33s/it]/home/users/jhischke/miniforge3/envs/artifact-detection/lib/python3.13/site-packages/mne_features/univariate.py:239: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return stats.skew(data, axis=ndim - 1)
/home/users/jhischke/miniforge3/envs/artifact-detection/lib/python3.13/site-packages/mne_features/univariate.py:258: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return stats.kurtosis(data, axis=ndim - 1, fisher=False)
Calculating features for sessions: 100%|██████████| 268/268 [18:51<00:00,  4.22s/it]


Diese vier Features haben 18 Minuten und 51 gedauert.

In [4]:
from utils.persistence import copy_labels

hdf5_feat = './features/features.hdf5'
hdf5_data = './labelled_data/labels.hdf5'

copy_labels(hdf5_data, hdf5_feat)

Copying labels in sessions: 100%|██████████| 268/268 [00:24<00:00, 10.75it/s]


Es ist kein Fehler aufgekommen. Nachfolgend ein Sanity check, ob die Zuordnung von Labels und den dazugehörigen Mittelwerten geändert wurde.

In [11]:
from tqdm import tqdm
import numpy as np

with h5py.File(hdf5_feat, 'r') as sf, h5py.File(hdf5_data, 'r') as df:
        for session in tqdm(sf, desc="Comparing labels in sessions"):
            for channel in sf[session]:
                if (sf[session][channel]['labels'][:] != df[session][channel]['labels'][:]).any() or (sf[session][channel]['mean'] [:] != np.mean(df[session][channel]['data_windows'][:], axis=1)).any():
                    print("Error")

Comparing labels in sessions: 100%|██████████| 268/268 [04:49<00:00,  1.08s/it]


Der Sanity Check war erfolgreich. In der nachfolgenden Zelle wird `pow_freq_bands` ausgerechnet, was gleichzeitig testet, ob die Änderung an der `calculate_features` Funktion, dass Features mit mehr als einer Zahl je Fenster als Matrix abgespeichert werden.

In [ ]:
from utils.training import calculate_features

features = ['pow_freq_bands', 'hurst_exp', 'decorr_time']
hdf5_feat = './features/features.hdf5'
hdf5_data = './labelled_data/labels.hdf5'

calculate_features(hdf5_feat, hdf5_data, features)

Calculating features for sessions:   0%|          | 0/268 [00:00<?, ?it/s]

Calculating features for sessions:   3%|▎         | 8/268 [03:55<1:19:51, 18.43s/it]

Leider ist die Konsolenausgabe abgebrochen, weshalb ich keine Zeit sehen kann, aber die Features sind dennoch da. Die geschätzte Zeit lag bei etwa 80 Minuten.

Nachfolgend werden die selben Zellen nochmal für die Daten ohne Hochpass durchgeführt.

In [1]:
from utils.training import calculate_features

features = ['mean', 'variance', 'std']
hdf5_feat = './features/features_no_hp.hdf5'
hdf5_data = './labelled_data/labels_no_hp.hdf5'

calculate_features(hdf5_feat, hdf5_data, features)

Calculating features for sessions: 100%|██████████| 268/268 [06:20<00:00,  1.42s/it]


In [2]:
import h5py
import numpy as np

with h5py.File(hdf5_feat) as f, h5py.File(hdf5_data) as df:
    print(f["aaaaaall_s004_t000"]['A1-T3']['mean'][:])
    print(np.mean(df["aaaaaall_s004_t000"]['A1-T3']['data_windows'], axis=1))

[ 0.4000448   4.56338298  4.38157372 ... -3.87476247 -9.89572936
 -3.54941435]
[ 0.4000448   4.56338298  4.38157372 ... -3.87476247 -9.89572936
 -3.54941435]


Sanity Check: bestanden

In [3]:
from utils.training import calculate_features

features = ['ptp_amp', 'skewness', 'kurtosis', 'quantile']
hdf5_feat = './features/features_no_hp.hdf5'
hdf5_data = './labelled_data/labels_no_hp.hdf5'

calculate_features(hdf5_feat, hdf5_data, features)

Calculating features for sessions:   2%|▏         | 5/268 [00:11<09:52,  2.25s/it]/home/users/jhischke/miniforge3/envs/artifact-detection/lib/python3.13/site-packages/mne_features/univariate.py:239: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return stats.skew(data, axis=ndim - 1)
/home/users/jhischke/miniforge3/envs/artifact-detection/lib/python3.13/site-packages/mne_features/univariate.py:258: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return stats.kurtosis(data, axis=ndim - 1, fisher=False)
Calculating features for sessions: 100%|██████████| 268/268 [09:10<00:00,  2.06s/it]


In [4]:
from utils.persistence import copy_labels

hdf5_feat = './features/features_no_hp.hdf5'
hdf5_data = './labelled_data/labels_no_hp.hdf5'

copy_labels(hdf5_data, hdf5_feat)

Copying labels in sessions: 100%|██████████| 268/268 [00:04<00:00, 62.61it/s]


In [5]:
from tqdm import tqdm
import numpy as np

with h5py.File(hdf5_feat, 'r') as sf, h5py.File(hdf5_data, 'r') as df:
        for session in tqdm(sf, desc="Comparing labels in sessions"):
            for channel in sf[session]:
                if (sf[session][channel]['labels'][:] != df[session][channel]['labels'][:]).any() or (sf[session][channel]['mean'] [:] != np.mean(df[session][channel]['data_windows'][:], axis=1)).any():
                    print("Error")

Comparing labels in sessions: 100%|██████████| 268/268 [01:05<00:00,  4.10it/s]


Sanity Check: bestanden

In [6]:
from utils.training import calculate_features

features = ['pow_freq_bands', 'hurst_exp', 'decorr_time']
hdf5_feat = './features/features_no_hp.hdf5'
hdf5_data = './labelled_data/labels_no_hp.hdf5'

calculate_features(hdf5_feat, hdf5_data, features)

Calculating features for sessions:   2%|▏         | 5/268 [03:30<2:59:53, 41.04s/it]/home/users/jhischke/miniforge3/envs/artifact-detection/lib/python3.13/site-packages/mne_features/univariate.py:383: RuntimeWarning: divide by zero encountered in log
  reg[j] = np.log(np.mean(_rs))
/home/users/jhischke/miniforge3/envs/artifact-detection/lib/python3.13/site-packages/mne_features/univariate.py:727: RuntimeWarning: invalid value encountered in divide
  pow_freq_bands = np.divide(pow_freq_bands,
/home/users/jhischke/miniforge3/envs/artifact-detection/lib/python3.13/site-packages/mne_features/univariate.py:72: RuntimeWarning: invalid value encountered in divide
  return acf[:(lags + 1)] / acf[0]
/home/users/jhischke/miniforge3/envs/artifact-detection/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/users/jhischke/miniforge3/envs/artifact-detection/lib/python3.13/site-packages/numpy/_co

Das Berechnen dieser drei Features dauert 2:48:52.